# DTMF

这是陈硕写的《数字信号处理入门实验》的第三个实验，介绍双音多频（DTMF）信号的解码。
最新版网址： http://github.com/chenshuo/notes

本章内容的视频讲解在
* [DTMF](https://www.youtube.com/watch?v=nwtPnGi7cFU)  国内：https://www.bilibili.com/video/BV1FB4y1t7D7
* [Goertzel 算法解码 DTMF](https://www.youtube.com/watch?v=0JdW0RXH9ik)  国内：https://www.bilibili.com/video/BV19g411C7T4

DTMF 是最简单的频率分析：判断信号是否保护某一特定频率成分。



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal

import librosa as rosa
import librosa.display
from IPython.display import Audio

from ipywidgets import interact

np.set_printoptions(suppress=True)

一般来说，

这是一幅流传很广的示意图：

![frequency](data/frequency_view.png)

By Phonical – Own work, CC BY-SA 4.0,
https://commons.wikimedia.org/w/index.php?curid=64473578

## 信号的合成（synthesis）与分解（analysis）

In [ ]:
w = np.linspace(0, 2*np.pi, 100)
x = np.sin(w)
x2 = np.sin(2*w)
x3 = np.sin(3*w)
p=plt.plot(w, x, w, x2, w, x3)

In [ ]:
x5 = np.sin(5*w)
x7 = np.sin(7*w)
plt.plot(w, x + x3 / 3 + x5 / 5 + x7 / 7)

基本问题：信号合成的求逆。例如对于采用率为 48kHz，合成一个 440Hz 的正弦信号：

$$x = \sin(2\pi f t), \quad \mathrm{where}\ f = 440\mathrm{Hz}$$

In [ ]:
fs = 48000
t = np.arange(fs * 10 / 1000) / fs  # 10ms
f = 440  # https://en.wikipedia.org/wiki/A440_(pitch_standard)
x0 = np.sin(2*np.pi*f*t)
plt.plot(t, x0)

In [ ]:
x1 = rosa.tone(440, sr=48000, duration=3)
plt.plot(x1[0:480])
print(sum(np.abs(x0-x1[0:480])))
Audio(data=x1, rate=48000)

如果拿到的是这个输入信号，如何求出它的频率、幅度、相位？

对于单一频率的信号，可以测量其周期（过零点），那么多频率的信号呢？

如果一个 1000Hz 的正弦波和一个 440Hz 的正弦波叠加在一起，得到信号 $y = \sin(1000 \omega) + \sin(440 \omega)$ . 如何找到这两个频率？

In [ ]:
x2 = rosa.tone(1000, sr=48000, duration=3)
y = x1 + x2
plt.plot(t, y[0:480])
Audio(data=y, rate=48000)

如果已知某信号包含高低两个频率，如何去掉其中任何一个？ Filtering

## DTMF

https://en.wikipedia.org/wiki/Dual-tone_multi-frequency_signaling

|        | 1209 Hz | 1336 Hz | 1477 Hz |
| -----: | :-----: | :-----: | :-----: |
| 697 Hz |  **1**  |  **2**  |  **3**  |
| 770 Hz |  **4**  |  **5**  |  **6**  |
| 852 Hz |  **7**  |  **8**  |  **9**  |
| 941 Hz |  **$*$** |  **0**  |  **#**  |

https://www.mathworks.com/help/signal/ug/dft-estimation-with-the-goertzel-algorithm.html

https://hackaday.com/2020/11/13/dsp-spreadsheet-the-goertzel-algorithm-is-fouriers-simpler-cousin/

2012 年 8 月”南京大学学生听拨号声破解周鸿祎手机号“


In [ ]:
import itertools

dtmf_row = [697, 770, 852, 941]
dtmf_col = [1209, 1336, 1477]
dtmf_digits = '123456789*0#'
digit_to_freq = dict(zip(dtmf_digits, itertools.product(dtmf_row, dtmf_col)))
digit_to_freq

## Generate DTMF tone


In [ ]:
digit_to_freq = {
    '1' : (697, 1209),
    '2' : (697, 1336),
    '3' : (697, 1477),
    '4' : (770, 1209),
    '5' : (770, 1336),
    '6' : (770, 1477),
    '7' : (852, 1209),
    '8' : (852, 1336),
    '9' : (852, 1477),
    '*' : (941, 1209),
    '0' : (941, 1336),
    '#' : (941, 1477),
}

fs = 8000
def dtmf_single(digit):
  on = 800   # 100ms
  x = np.zeros(on)
  w = np.arange(on) / fs * 2 * np.pi
  freqs = digit_to_freq[digit]
  for f in freqs:
    x += np.sin(w * f) * 0.5
  return x

In [ ]:
digit_1 = dtmf_single('1')
plt.plot(digit_1[0:100])
Audio(digit_1, rate=fs)

In [ ]:
digit_2 = dtmf_single('2')
plt.plot(digit_2[0:100])
Audio(digit_2, rate=fs)

In [ ]:
def dtmf_multi(digits):
  off = 800  # 100ms
  gap = np.zeros(off)
  y = np.array([])
  for d in digits:
    if d != '-':
      x = dtmf_single(d)
    else:
      x = np.zeros(200)
    y = np.append(y, x)
    y = np.append(y, gap)
  return y

In [ ]:
dial_e = dtmf_multi('271-828-1828')
plt.plot(dial_e)
Audio(dial_e, rate=fs)

In [ ]:
dial_today = dtmf_multi('2022-08-31')
Audio(dial_today, rate=fs)

## Single point DFT


一个 Naive 的频率分析方法：用已知频率的正弦信号去点乘输入信号（计算相关度），如果得到的数越大，说明输入信号在这个频率的分量越大。这正是 DFT/FFT 的原理（之一）。

$$X[k] = \sum_{n=0}^{N-1} x[n] e^{-j 2 \pi  n k / N}$$

In [ ]:
def single_dft(x, k):
  N = len(x)
  n = np.arange(N) / N  #  n in 0, 1, ..., N-1
  w = np.exp(-2j * np.pi * n * k)
  return np.dot(x, w)
  # Horner's rule to save space, if necessary

## Find $N$ and $k$

For Fs = 8000

* N = 205 in "Add DTMF generation and decoding to DSP-μP designs", Pat Mock, 1989.
https://www.ti.com/litv/pdf/spra168

* N = 105 in "Modified Goertzel Algorithm in DTMF Detection Using the TMS320C80", Chiouguey J. Chen, 1996. TI SPRA066

* N = 136 in "DTMF Tone Generation and Detection: An Implementation Using the TMS320C54x", Gunter Schmer, 2000. TI SPRA096A.


In [ ]:
freqs = np.array([697, 770, 852, 941, 1209, 1336, 1477])
np.diff(freqs)

In [ ]:
np.round(freqs * 21 / 19, 1)

https://engineering.stackexchange.com/questions/37693/how-were-the-tones-for-dtmf-chosen
> The tones have been carefully selected to minimize harmonic interference and the probability that a pair of high and low tones will be simulated by the human voice, thus protecting network control signaling.

In [ ]:
freqs * 2

In [ ]:
@interact(nfft = (60, 250, 5))
def how_many_bins(nfft=205):
  fs = 8000
  t = np.arange(nfft)/fs
  x = np.sin(2*np.pi* freqs.reshape(len(freqs), 1) * t)
  print('bin = %.2f Hz' % (fs / nfft))
  for i in x:
    fft = np.fft.rfft(i)
    plt.plot(np.abs(fft[0:(nfft//4)])/nfft)

Find k's

In [ ]:
N = 205
bin = fs / N
print(freqs / bin)
np.round(freqs / bin, 0)

In [ ]:
N = 205
sr = 8000
print('%.2f Hz' % (sr / N))

k = np.array([18, 20, 22, 24, 31, 34, 38])
print(np.round(k * sr / N, 2))
print(np.round(k * sr / N - freqs, 2))

In [ ]:
N = 205
x = dtmf_single('7')[0:N]

k = 31
fft = np.fft.rfft(x)
print('fft:', fft[k])

dft = single_dft(x, k)
print('dft:', dft, np.abs(dft))
print('diff:', np.abs(dft - fft[k]))

In [ ]:
N = 205
x = dtmf_single('8')[0:N]

bins = [18, 20, 22, 24, 31, 34, 38]
y = np.zeros(len(bins))

for i, k in enumerate(bins):
  y[i] = np.abs(single_dft(x, k))
plt.figure(figsize=(15,5))
plt.subplot(121)
plt.stem(bins, y, use_line_collection=True)
plt.xlim(16, 39)
plt.subplot(122)
plt.plot(x)

In [ ]:
@interact(digit=(0, 9, 1))
def dtmf(digit=1):
  N = 205
  x = dtmf_single(str(digit))[0:N]

  bins = [18, 20, 22, 24, 31, 34, 38]
  y = np.zeros(len(bins))

  for i, k in enumerate(bins):
    y[i] = np.abs(single_dft(x, k))
  plt.figure(figsize=(15,5))
  plt.subplot(121)
  plt.stem(bins, y, use_line_collection=True)
  plt.xlim(15, 40)
  plt.ylim(-5, 60)
  plt.subplot(122)
  plt.plot(x[0:100])

Threshold = 30

In [ ]:
N = 205
x = np.zeros(N)
x[0] = 1
bins = [18, 20, 22, 24, 31, 34, 38]
for k in bins:
  print(np.abs(single_dft(x, k)))

## Goertzel algorithm

格尔泽 1958, earlier than FFT.

https://en.wikipedia.org/wiki/Goertzel_algorithm

A Simpler Goertzel Algorithm, Rick Lyons, 2021.
https://www.dsprelated.com/showarticle/1386.php

![Goertzel](data/goertzel.png)

https://www.mathworks.com/help/signal/ug/dft-estimation-with-the-goertzel-algorithm.html

$\omega = \dfrac{2\pi k} {N}$

Loop through x[n]: $s[n]=x[n]+2\cos(\omega)s[n-1]-s[n-2]$

At the end, $X(\omega) =  e^{j\omega}s[N-1] -s[N-2]$

$power = (s[N-1])^2 + (s[N-2])^2 - 2 \cos(\omega)s[N-1]s[N-2]$

* one coefficient: $2\cos \omega$
* real number arithmetic
  * $N$ multiplications and $2N$ additions for $s[N]$
* constant space (2 real numbers for $s[n-1]$ and $s[n-2]$)

In [ ]:
k = 18
N = 205
x = dtmf_single('1')[0:N]

w = k * 2 * np.pi / N
c = np.cos(w)
c2 = 2 * c
s1 = 0
s2 = 0

for p in x:
  s0 = p + c2 * s1 - s2
  s2 = s1
  s1 = s0

goert = np.exp(1j*w) * s1 - s2

dft = single_dft(x, k)
print(dft) 
print(goert)
print(np.real_if_close(dft-goert))

power = s1 * s1 + s2 * s2 - c2 * s1 * s2

print(np.abs(dft) ** 2)
print(np.abs(goert) ** 2)
print(power)

In [ ]:
def goertzel(x, k):
  N = len(x)
  s2 = 0
  s1 = 0
  s0 = 0
  w = k * 2 * np.pi / N
  c2 = 2 * np.cos(w)
  for p in x:
    s0 = p + c2 * s1 - s2
    s2 = s1
    s1 = s0
  power = s1 * s1 + s2 * s2 - c2 * s1 * s2
  return power

In [ ]:
N = 205
bins = [18, 20, 22, 24, 31, 34, 38]

results = np.zeros((10, len(bins)))

for digit in '0123456789':
  x = dtmf_single(digit)[0:N]
  fft = np.fft.rfft(x)
  for i, k in enumerate(bins):
    g = goertzel(x, k)
    assert np.allclose(g, np.abs(fft[k])**2)
    results[int(digit), i] = g
print(np.round(results, 0))

power threshold = 1000

## Detect single digit

In [ ]:
bin_to_digit = {}
N = 205
fs = 8000

for digit, freqs in digit_to_freq.items():
  bins = np.round(np.array(freqs) / (fs / N), 0)
  key = (int(bins[0]), int(bins[1]))
  bin_to_digit[key] = digit
print(bin_to_digit)

# return '0' ~ '9' if detected, - if not
def detect_single(x):
  N = 205
  if len(x) < N:
    x = np.append(x, np.zeros(N - len(x)))
  x = x[0:N]
  assert len(x) == N

  bins = [18, 20, 22, 24, 31, 34, 38]
  found = []
  for k in bins:
    g = goertzel(x, k)
    if g > 1000:
      found.append(k)
  return bin_to_digit.get(tuple(found), '-')

In [ ]:
for digit in '0123456789':
  print(digit, detect_single(dtmf_single(digit)))

print(detect_single(np.zeros(0)))
print(detect_single(np.zeros(100)))
print(detect_single(np.zeros(500)))
print(detect_single(np.ones(1000)))
print(detect_single(dtmf_single('1') + dtmf_single('2')))
print(detect_single(dtmf_single('1') + dtmf_single('5')))
print(detect_single(dtmf_single('1')*0.5 + dtmf_single('6')))

## Put it together

In [ ]:
def dtmf_decode(x):
  result = ''
  i = 0
  N = 205

  last_digit = '-'
  last_count = 0
  while i < len(x):
    digit = detect_single(x[i:i+N])
    i += N
    if digit == last_digit:
      last_count += 1
      continue
    if digit == '-':
      if last_count >= 2:
        result += last_digit
    last_digit = digit
    last_count = 1
  return result

In [ ]:
print(dtmf_decode(dtmf_multi('123-456-7890')))

Online DTMF Tone Generator: 
https://www.audiocheck.net/audiocheck_dtmf.php

In [ ]:
x, sr = rosa.load('data/dtmf_2022_07_31.wav', sr=None)
assert sr == 8000
plt.plot(x)
Audio(x, rate=sr, normalize=False)

In [ ]:
print(dtmf_decode(x))

https://en.wikipedia.org/wiki/File:DTMF_dialing.ogg

In [ ]:
x, sr = rosa.load('data/DTMF_dialing.ogg', sr=None)
assert sr == 8000
plt.plot(x)
Audio(x, rate=sr, normalize=False)

In [ ]:
y = dtmf_decode(x)
print(len(y), y)

In [ ]:
print(len('06966753564646415180233673141636083381604400826146625368963884821381785073643399'))

In [ ]:
print(75/80)

https://www.sigidwiki.com/wiki/Dual_Tone_Multi_Frequency_(DTMF)

## Real world requirements

https://web.archive.org/web/20110925184759/http://nemesis.lonestar.org/reference/telecom/signaling/dtmf.html

* Accept frequency error < 1.5%
* Reject frequency error > 3.5%
* Duration 50ms/45ms
* Power 0 to -25 dBm, 
* Twist 

Some real world DTMF recievers also checks 2nd harmonics of DTMF frequencies.
So 16 points are calculated.

DTMF receiver chips:

* CM8870CSI (obsoleted) by California Micro Devices Corp. 215 Topaz Street, Milpitas, California  95035
* MT8870D from Zarlink (active)

![MT8870D](data/mt8870d.jpg)

Real world code:

https://github.com/freeswitch/spandsp/blob/master/src/dtmf.c

## Summary

Goertzel Algorithm for a Non-integer Frequency Index, Rick Lyons, 2013.
https://www.dsprelated.com/showarticle/495.php

### Learn a neural network for DTMF detection?

![NN](data/dtmf-nn.png)

* First layer: 7 neuros, each to detect a frequency
  * DFT can be viewed as a signle layer NN, no bais, no activiation function, just particular weights.
* Second layer: 12 neuros, simple AND function, to output 0~9 and * and #.
* Find best N using grid searching ?